Read in the training set

In [1]:
import pandas as pd
#Use this line of code for the clean set
Train = pd.read_csv("QQP/train.tsv",sep="\t")
#Use thie line of code for the original set
#Train = pd.read_csv("QQP/train.tsv")
#Use this line when we just want to train on a subset of the training set.
#Train = Train.iloc[1:10000,]



Pre-Processing of the training set

In [2]:
#1. Lowercase all questions and remove punctuation

#Remark: We need to remove puntations before removing empty columns, because some questions only contains puntuations and should be considered missing values. However, they will not be dropped if the puntutations are present when we executed the drop command

##To remove punctuation: We will use a code provided by CHATGPT

import string

def remove_punctuation(text):
    # Create a translation table to remove punctuation characters
    translator = str.maketrans('', '', string.punctuation)

    # Use the translation table to remove punctuation
    text_without_punctuation = text.translate(translator)

    return text_without_punctuation

Train = Train.dropna()
Train = Train[~Train.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

#For each question: Set all to lowercase then remove puntuation, and replace it in the original Train dataframe

for i in range(len(Train["question1"])):
    #print(i)
    Train["question1"].iloc[i] = remove_punctuation(Train["question1"].iloc[i].lower())
    Train["question2"].iloc[i] = remove_punctuation(Train["question2"].iloc[i].lower())

#2. Remove rows with missing data
Train = Train.dropna()
Train = Train[~Train.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

#2. Extract the questions
##Extract the first questions and store it in a variable
Q1 = Train["question1"].copy()
##Extract the second questions and store it in a variable
Q2 = Train["question2"].copy()





/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
Train["question2"].iloc[105780]

'how do i create a successful app'

Tokenize the questions in each question pair

In [4]:
##Create a copy of each of the questions
Q1_Tokens = Q1.copy()
Q2_Tokens = Q2.copy()
#For each question:
for i in range(len(Q1_Tokens)):
    #Tokenize each question and overwrite each question with its token
    Q1_Tokens.iloc[i] = Q1_Tokens.iloc[i].split()
    Q2_Tokens.iloc[i] = Q2_Tokens.iloc[i].split()

Create embeddings using Word2Vec

In [5]:
import gensim.downloader
Embedder = gensim.downloader.load('word2vec-google-news-300')

Create embeddings for training set

In [6]:
import numpy as np
from gensim.models import Word2Vec
Q1_Embedded = Q1_Tokens.copy()
Q2_Embedded = Q2_Tokens.copy()
#For each question:
for i in range(len(Q1_Embedded)):
  Sum_Q1 = np.zeros(300)
  #For each word in the ith question:
  for word in Q1_Embedded.iloc[i]:
    #Check if the word is in our pre-trained model
    if word in Embedder:
      #Find the embedding and add it to the running sum
      Sum_Q1 = Sum_Q1 + Embedder["word"]
  #After we have summed up the vectors for all the words in the ith questions, take the average and store the resultant vector
  Q1_Embedded.iloc[i] = Sum_Q1 / len(Q1_Tokens.iloc[i])
  #Repeat the same for the other question in this pair
  Sum_Q2 = np.zeros(300)
  for word in Q2_Embedded.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + Embedder["word"]
  Q2_Embedded.iloc[i] = Sum_Q2 / len(Q2_Tokens.iloc[i])



Create embeddings for test set

In [7]:
#Test = pd.read_csv("QQP/dev.tsv")
Test = pd.read_csv("QQP/dev.tsv",sep="\t")
Test = Test.dropna()
Test = Test[~Test.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Test["question1"])):
    Test["question1"].iloc[i] = remove_punctuation(Test["question1"].iloc[i].lower())
    Test["question2"].iloc[i] = remove_punctuation(Test["question2"].iloc[i].lower())

#2. Remove rows with missing data

#2. Remove rows with missing data
Test = Test.dropna()
Test = Test[~Test.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

#2. Extract the questions
##Extract the first questions and store it in a variable
Q1_Test = Test["question1"].copy()
##Extract the second questions and store it in a variable
Q2_Test = Test["question2"].copy()


##Create a copy of each of the questions
Q1_Test_Tokens = Q1_Test.copy()
Q2_Test_Tokens = Q2_Test.copy()
#For each question:
for i in range(len(Q1_Test_Tokens)):
    #Tokenize each question and overwrite each question with its token
    Q1_Test_Tokens.iloc[i] = Q1_Test_Tokens.iloc[i].split()
    Q2_Test_Tokens.iloc[i] = Q2_Test_Tokens.iloc[i].split()

from gensim.models import Word2Vec
Q1_Embedded_Test = Q1_Test_Tokens.copy()
Q2_Embedded_Test = Q2_Test_Tokens.copy()


#For each question:
for i in range(len(Q1_Embedded_Test)):
  Sum_Q1 = np.zeros(300)
  #For each word:
  for word in Q1_Embedded_Test.iloc[i]:
    if word in Embedder:
      #Create the embedding and store it inside of the list
      Sum_Q1 = Sum_Q1 + (Embedder[word])
  Q1_Embedded_Test.iloc[i] = Sum_Q1 / len(Q1_Test_Tokens.iloc[i])
  #Create a list that will contain the vector embeddings for each word in the question
  Sum_Q2 = np.zeros(300)
  #For each word:
  for word in Q2_Embedded_Test.iloc[i]:
    if word in Embedder:
      #Create the embedding and store it inside of the list
      Sum_Q2 = Sum_Q2 + (Embedder[word])
  Q2_Embedded_Test.iloc[i] = Sum_Q2 / len(Q2_Test_Tokens.iloc[i])

#Clear up ram as we don't need the embedder anymore
Embedder = 0

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Aggregate each question to get a vector representation of each question, using the average method

Initialization of what is the underlying model we will use in the Siamese architecture


In [8]:
#Creating a multi-layer perceptron neural network that will be used as the underlying architecture

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import cosine_similarity

#Identify the length of the vectors used in the embeddings
embedding_dim = 300

#Identify the number of questions. This will indicate how many nodes we will use
Questions_Count = Q2_Embedded.shape[0]

#This indicates that we will be using 4 layers in our underlying architecture. ~ Can be increased/decreased
shared_network = keras.Sequential([
    layers.Dense(300, activation='relu', input_shape=(embedding_dim,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid'),

])


Initialize the Siamese network architecture with the above model as the underlying architecture

In [9]:
#Initialize the Siamese network architecture

#Define the left and right inputs for the question pair ~ Initializing how long the vectors that represent each question is
left_input = layers.Input(shape=(embedding_dim,))
right_input = layers.Input(shape=(embedding_dim,))

# Encode the question pair using the shared network ~ Indicate that we will input the questions both question in the pair into the shared network
encoded_left = shared_network(left_input)
encoded_right = shared_network(right_input)

# Calculate the Euclidean distance between the encodings ~ I.e: We will use this distance function to determine the similarity between the outputs
distance = layers.Lambda(lambda x: tf.norm(x[0] - x[1], axis=1, keepdims=True))([encoded_left, encoded_right])
#distance=layers.Lambda(lambda x: cosine_similarity(x[0], x[1]))([encoded_left, encoded_right])

# We have initialized the model. Now, just create the Siamese model
siamese_model = keras.Model(inputs=[left_input, right_input], outputs=distance)


#Compile the model ~ Specifies how the training should be done etc.....
siamese_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#siamese_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


Training of the model

In [10]:
#Inputs has to be an array of a list of lists -> Convert from array of list to a list

Q1_Inputs = []

for i in range(len(Q1_Embedded)):
    #print(i)
    Q1_Inputs.append(Q1_Embedded.iloc[i].tolist())

Q1_Inputs = np.array(Q1_Inputs)

Q2_Inputs = []

for i in range(len(Q2_Embedded)):
    Q2_Inputs.append(Q2_Embedded.iloc[i].tolist())

Q2_Inputs = np.array(Q2_Inputs)


#Training of the model

siamese_model.fit(
    [Q1_Inputs, Q2_Inputs],  # Your question embeddings
    np.array(Train["is_duplicate"]),  # Similarity labels (0 for dissimilar, 1 for similar)
    batch_size=32,
    epochs=10,
    validation_split=0.2  # You can adjust the validation split
)



Epoch 1/10
9096/9096 [==============================] - 41s 4ms/step - loss: 2.4197 - accuracy: 0.5962 - val_loss: 2.4196 - val_accuracy: 0.5932
Epoch 2/10
9096/9096 [==============================] - 41s 5ms/step - loss: 2.4104 - accuracy: 0.5924 - val_loss: 2.4224 - val_accuracy: 0.5828
Epoch 3/10
9096/9096 [==============================] - 40s 4ms/step - loss: 2.4093 - accuracy: 0.5913 - val_loss: 2.4193 - val_accuracy: 0.5942
Epoch 4/10
9096/9096 [==============================] - 41s 5ms/step - loss: 2.4089 - accuracy: 0.5915 - val_loss: 2.4172 - val_accuracy: 0.5909
Epoch 5/10
9096/9096 [==============================] - 41s 5ms/step - loss: 2.4088 - accuracy: 0.5912 - val_loss: 2.4176 - val_accuracy: 0.5867
Epoch 6/10
9096/9096 [==============================] - 43s 5ms/step - loss: 2.4084 - accuracy: 0.5910 - val_loss: 2.4213 - val_accuracy: 0.5957
Epoch 7/10
9096/9096 [==============================] - 41s 4ms/step - loss: 2.4082 - accuracy: 0.5908 - val_loss: 2.4167 - val_ac

Finding the Training Accuracuy

In [11]:
train_predictions = siamese_model.predict([Q1_Inputs, Q2_Inputs])
threshold = 0.5
train_binary_predictions = (train_predictions > threshold).astype(int)
from sklearn.metrics import accuracy_score
training_accuracy = accuracy_score(Train["is_duplicate"].tolist(), train_binary_predictions)
print(f"Training Accuracy: {training_accuracy * 100:.2f}%")


11370/11370 [==============================] - 12s 1ms/step
Training Accuracy: 58.87%


In [12]:
print(sum(train_binary_predictions))

[31256]


In [13]:
Q1_Inputs_Test = []

for i in range(len(Q1_Embedded_Test)):
    Q1_Inputs_Test.append(Q1_Embedded_Test.iloc[i].tolist())

Q1_Inputs_Test = np.array(Q1_Inputs_Test)

Q2_Inputs_Test = []

for i in range(len(Q2_Embedded_Test)):
    Q2_Inputs_Test.append(Q2_Embedded_Test.iloc[i].tolist())

Q2_Inputs_Test = np.array(Q2_Inputs_Test)


test_predictions = siamese_model.predict([Q1_Inputs_Test, Q2_Inputs_Test])
threshold = 0.5
test_binary_predictions = (test_predictions > threshold).astype(int)
from sklearn.metrics import accuracy_score
test_accuracy = accuracy_score(Test["is_duplicate"].tolist(), test_binary_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


1264/1264 [==============================] - 2s 1ms/step
Test Accuracy: 63.12%
